## Imports

In [5]:
import pandas as pd
import umap
import umap.plot
import plotly.express as px
import numpy as np
import plotly.io
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.transform import linear_cmap
import matplotlib.pyplot as plt

output_notebook()

Loading BokehJS ...

In [6]:
embeddings_df = pd.read_csv('data/mut_escape_embeddings.csv')

In [7]:
mapper = umap.UMAP().fit(embeddings_df.loc[:, 'x_0':])

In [8]:
u = umap.UMAP().fit_transform(embeddings_df.loc[:, 'x_0':])

In [9]:
scores_df = pd.read_csv('data/aggregated_mut_escapes.csv')
scores_df.describe()

,site,2B04,2C02,2C03,2E06,COV2-2050,COV2-2082,COV2-2094,COV2-2096,COV2-2130,...,subject G (day 18),subject G (day 94),subject H (day 152),subject H (day 61),subject I (day 102),subject I (day 26),subject J (day 121),subject J (day 15),subject K (day 103),subject K (day 29)
count,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,...,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000,2366.000000
mean,431.281910,-1.254765,-1.285771,-1.225181,-1.272193,-0.018117,-0.019425,-0.017406,0.004595,-1.553122,...,-1.569014,-1.549485,-1.579584,-1.587125,-1.581097,-1.584351,-1.572326,-1.576829,-1.598747,-1.595780
std,60.380775,3.135842,3.160975,3.150349,3.139245,0.453775,0.491140,0.453830,0.188634,3.443650,...,3.443485,3.452044,3.448250,3.444600,3.463081,3.461367,3.466465,3.439432,3.444129,3.440527
min,331.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,...,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000
25%,374.000000,0.001029,0.002636,0.001268,0.000276,0.000958,0.002381,0.001233,0.001733,0.000595,...,0.001862,0.029210,0.002892,0.000000,0.005570,0.000992,0.025087,0.002905,0.000711,0.000000
50%,440.000000,0.001977,0.003879,0.001318,0.000414,0.001419,0.002803,0.001877,0.001809,0.000930,...,0.003204,0.042975,0.005263,0.003077,0.005571,0.003861,0.035135,0.004973,0.001409,0.000364
75%,482.000000,0.003866,0.004766,0.003296,0.000494,0.002024,0.003191,0.002873,0.001921,0.001875,...,0.008764,0.054655,0.011575,0.010470,0.007489,0.011067,0.046238,0.012417,0.002387,0.001262
max,531.000000,0.621700,0.511300,0.758600,0.427400,0.378600,1.000000,0.516400,0.457200,0.993000,...,0.564100,0.164700,0.245600,0.181300,0.528200,0.453800,0.277100,0.308400,0.198500,0.096160


In [10]:
criteria = scores_df.loc[:, '2B04':] >= 0.0
print(criteria)

       2B04   2C02   2C03   2E06  COV2-2050  COV2-2082  COV2-2094  COV2-2096  \
0      True   True   True   True       True       True       True       True   
1      True   True   True   True       True       True       True       True   
2      True   True   True   True       True       True       True       True   
3      True   True   True   True       True       True       True       True   
4      True   True   True   True       True       True       True       True   
...     ...    ...    ...    ...        ...        ...        ...        ...   
2361  False  False  False  False       True       True       True       True   
2362  False  False  False  False      False       True      False       True   
2363  False  False  False  False      False       True      False       True   
2364  False  False  False  False      False      False      False       True   
2365  False  False  False  False      False      False      False       True   

      COV2-2130  COV2-2165  ...  subjec

In [38]:
conditions = scores_df.columns[3:]

from bokeh.palettes import magma
from bokeh.palettes import Magma

for condition in conditions:
    # Get the number of colors we'll need for the plot.
    colors = magma(256)

    # Create a map between factor and color.
    colormap = dict()
    for x in scores_df[condition].unique():
        colormap[x] = colors[int(max(x, 0) * 255)]

    # Create a list of colors for each value that we will be looking at.
    colors = [colormap[x] for x in scores_df[condition]]

    p = figure(title=condition, plot_width=1024, plot_height=1024)
    p.circle(
        u[:, 0], 
        u[:, 1], 
        # palette='Cividis',
        color=colors,
        alpha=np.maximum(scores_df[condition], 0.0) + 0.05,
        size=(np.maximum(scores_df[condition], 0.0) + 1) * 5,
    )

    p.vbar(x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
           top = [9] * 11, 
           bottom = [8] * 11, 
           width = 1, 
           color = Magma[11])     
    show(p)
    break

AttributeError: unexpected attribute 'y' to VBar, possible attributes are bottom, fill_alpha, fill_color, hatch_alpha, hatch_color, hatch_extra, hatch_pattern, hatch_scale, hatch_weight, js_event_callbacks, js_property_callbacks, line_alpha, line_cap, line_color, line_dash, line_dash_offset, line_join, line_width, name, subscribed_events, tags, top, width or x

In [28]:
conditions = scores_df.columns[3:]

for condition in conditions:
    fig = px.scatter(
        u, x=0, y=1, color=np.maximum(scores_df[condition], 0.0), title=condition, size=np.maximum(scores_df[condition], 0.0)
    )
    fig.show()
    break

In [ ]:
umap.plot.points(mapper)